In [27]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os
import gc

import random
from datetime import datetime
from time import time
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from box import Box

import warnings

warnings.filterwarnings(action='ignore')

In [28]:
# 메모리 정리
gc.collect()
torch.cuda.empty_cache()

# 1. 학습 설정

In [29]:
config = {
    'data_path' : "/opt/ml/input/data/train" , # 데이터 경로
    
    'submission_path' : "../submission",
    'submission_name' : 'LightGCN_submission.csv', 

    'model_path' : "../model", # 모델 저장 경로
    'model_name' : 'LightGCN_v1.pt',

    'num_epochs' : 50,
    "reg" : 1e-5,
    'lr' : 0.0001,
    "emb_dim" : 512,
    "n_layers" : 3,
    'batch_size' : 1024,
    "node_dropout" : 0.2,

    'valid_samples' : 10, # 검증에 사용할 sample 수
    'seed' : 22,
    'n_batch' : 30,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

In [30]:
if not os.path.isdir(config.model_path):
    os.mkdir(config.model_path)

In [31]:
if not os.path.isdir(config.submission_path):
    os.mkdir(config.submission_path)

# 2. 데이터 전처리

In [32]:
class MakeGraphDataSet():
    """
    GraphDataSet 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, 'train_ratings.csv'))

        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('item')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('user')
        self.num_item, self.num_user = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['item'].apply(lambda x : self.item_encoder[x])
        self.df['user_idx'] = self.df['user'].apply(lambda x : self.user_encoder[x])
        
        self.exist_users = [i for i in range(self.num_user)]
        self.exist_items = [i for i in range(self.num_item)]
        self.user_train, self.user_valid = self.generate_sequence_data()
        self.R_train, self.R_valid, self.R_total = self.generate_dok_matrix()
        self.ngcf_adj_matrix = self.generate_ngcf_adj_matrix()
        self.n_train = len(self.R_train)
        self.batch_size = self.config.batch_size

    def generate_encoder_decoder(self, col : str) -> dict:
        """
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Returns:
            dict: 생성된 user encoder, decoder
        """

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id

        return encoder, decoder
    
    def generate_sequence_data(self) -> dict:
        """
        sequence_data 생성

        Returns:
            dict: train user sequence / valid user sequence
        """
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        for user, item, time in zip(self.df['user_idx'], self.df['item_idx'], self.df['time']):
            users[user].append(item)
        
        for user in users:
            np.random.seed(self.config.seed)

            user_total = users[user]
            valid = np.random.choice(user_total, size = self.config.valid_samples, replace = False).tolist()
            train = list(set(user_total) - set(valid))

            user_train[user] = train
            user_valid[user] = valid # valid_samples 개수 만큼 검증에 활용 (현재 Task와 가장 유사하게)

        return user_train, user_valid
    
    def generate_dok_matrix(self):
        R_train = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        R_valid = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        R_total = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        user_list = self.exist_users
        for user in user_list:
            train_items = self.user_train[user]
            valid_items = self.user_valid[user]
            
            for train_item in train_items:
                R_train[user, train_item] = 1.0
                R_total[user, train_item] = 1.0
            
            for valid_item in valid_items:
                R_valid[user, valid_item] = 1.0
                R_total[user, valid_item] = 1.0
        
        return R_train, R_valid, R_total

    def generate_ngcf_adj_matrix(self):
        adj_mat = sp.dok_matrix((self.num_user + self.num_item, self.num_user + self.num_item), dtype=np.float32)
        adj_mat = adj_mat.tolil() # to_list
        R = self.R_train.tolil()

        adj_mat[:self.num_user, self.num_user:] = R
        adj_mat[self.num_user:, :self.num_user] = R.T
        adj_mat = adj_mat.todok() # to_dok_matrix

        def normalized_adj_single(adj):
            rowsum = np.array(adj.sum(1))
            d_inv = np.power(rowsum, -.5).flatten()  
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)
            norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv)

            return norm_adj.tocoo()

        ngcf_adj_matrix = normalized_adj_single(adj_mat)
        return ngcf_adj_matrix.tocsr()

    def sampling(self):
        users = random.sample(self.exist_users, self.config.batch_size)

        def sample_pos_items_for_u(u, num):
            pos_items = self.user_train[u]
            pos_batch = random.sample(pos_items, num)
            return pos_batch
        
        def sample_neg_items_for_u(u, num):
            neg_items = list(set(self.exist_items) - set(self.user_train[u]))
            neg_batch = random.sample(neg_items, num)
            return neg_batch
        
        pos_items, neg_items = [], []
        for user in users:
            pos_items += sample_pos_items_for_u(user, 1)
            neg_items += sample_neg_items_for_u(user, 1)
        
        return users, pos_items, neg_items

    def get_train_valid_data(self):
        return self.user_train, self.user_valid

    def get_R_data(self):
        return self.R_train, self.R_valid, self.R_total

    def get_ngcf_adj_matrix_data(self):
        return self.ngcf_adj_matrix

# 3. 모델

In [33]:
class LightGCN(nn.Module):
    def __init__(self, n_users, n_items, emb_dim, n_layers, reg, node_dropout, adj_mtx):
        super().__init__()

        # initialize Class attributes
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.l = adj_mtx
        self.graph = self._convert_sp_mat_to_sp_tensor(self.l)

        self.reg = reg
        self.n_layers = n_layers
        self.node_dropout = node_dropout

        # Initialize weights
        self.weight_dict = self._init_weights()
        print("Weights initialized.")

    # initialize weights
    def _init_weights(self):
        print("Initializing weights...")
        weight_dict = nn.ParameterDict()

        initializer = torch.nn.init.xavier_uniform_
        
        weight_dict['user_embedding'] = nn.Parameter(initializer(torch.empty(self.n_users, self.emb_dim).to(device)))
        weight_dict['item_embedding'] = nn.Parameter(initializer(torch.empty(self.n_items, self.emb_dim).to(device)))
           
        return weight_dict

    # convert sparse matrix into sparse PyTorch tensor
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res

    # apply node_dropout
    def _droupout_sparse(self, X):
        """
        Drop individual locations in X
        
        Arguments:
        ---------
        X = adjacency matrix (PyTorch sparse tensor)
        dropout = fraction of nodes to drop
        noise_shape = number of non non-zero entries of X
        """
        node_dropout_mask = ((self.node_dropout) + torch.rand(X._nnz())).floor().bool().to(device)
        i = X.coalesce().indices()
        v = X.coalesce()._values()
        i[:,node_dropout_mask] = 0
        v[node_dropout_mask] = 0
        X_dropout = torch.sparse.FloatTensor(i, v, X.shape).to(X.device)

        return  X_dropout.mul(1/(1-self.node_dropout))

    def forward(self, u, i, j):
        """
        Computes the forward pass
        
        Arguments:
        ---------
        u = user
        i = positive item (user interacted with item)
        j = negative item (user did not interact with item)
        """
        # apply drop-out mask
        graph = self._droupout_sparse(self.graph) if self.node_dropout > 0 else self.graph
        ego_embeddings = torch.cat([self.weight_dict['user_embedding'], self.weight_dict['item_embedding']], 0)
        final_embeddings = [ego_embeddings]

        for k in range(self.n_layers):
            ego_embeddings = torch.sparse.mm(graph, final_embeddings[k])
            final_embeddings.append(ego_embeddings)                                       

        final_embeddings = torch.stack(final_embeddings, dim=1)
        final_embeddings = torch.mean(final_embeddings, dim=1)
        
        u_final_embeddings, i_final_embeddings = final_embeddings.split([self.n_users, self.n_items], 0)

        self.u_final_embeddings = nn.Parameter(u_final_embeddings)
        self.i_final_embeddings = nn.Parameter(i_final_embeddings)
        
        # loss 계산
        u_emb = u_final_embeddings[u] # user embeddings
        p_emb = i_final_embeddings[i] # positive item embeddings
        n_emb = i_final_embeddings[j] # negative item embeddings
        
        y_ui = torch.sum(torch.mul(u_emb, p_emb), dim = 1)                        
        y_uj = torch.sum(torch.mul(u_emb, n_emb), dim = 1)
        
        log_prob = torch.mean(torch.log(torch.sigmoid(y_ui - y_uj))) 
        bpr_loss = -log_prob        
        if self.reg > 0.:
            l2norm = (torch.sum(u_emb**2)/2. + torch.sum(p_emb**2)/2. + torch.sum(n_emb**2)/2.) / u_emb.shape[0]
            l2reg = self.reg * l2norm
            bpr_loss += l2reg

        
        return bpr_loss

# 4. 학습 함수

In [34]:
def train(model, make_graph_data_set, optimizer, n_batch):
    model.train()
    loss_val = 0
    for step in range(1, n_batch + 1):
        user, pos, neg = make_graph_data_set.sampling()
        optimizer.zero_grad()
        loss = model(user, pos, neg)
        loss.backward()
        optimizer.step()
        loss_val += loss.item()
    loss_val /= n_batch
    return loss_val

def split_matrix(X, n_splits=10):
    splits = []
    chunk_size = X.shape[0] // n_splits
    for i in range(n_splits):
        start = i * chunk_size
        end = X.shape[0] if i == n_splits - 1 else (i + 1) * chunk_size
        splits.append(X[start:end])
    return splits

def compute_ndcg_k(pred_items, test_items, test_indices, k):
    
    r = (test_items * pred_items).gather(1, test_indices)
    f = torch.from_numpy(np.log2(np.arange(2, k+2))).float().to(device)
    
    dcg = (r[:, :k]/f).sum(1)                                               
    dcg_max = (torch.sort(r, dim=1, descending=True)[0][:, :k]/f).sum(1)   
    ndcg = dcg/dcg_max                                                     
    
    ndcg[torch.isnan(ndcg)] = 0
    return ndcg

def evaluate(u_emb, i_emb, Rtr, Rte, k = 10):

    # split matrices
    ue_splits = split_matrix(u_emb)
    tr_splits = split_matrix(Rtr)
    te_splits = split_matrix(Rte)

    recall_k, ndcg_k= [], []
    # compute results for split matrices
    for ue_f, tr_f, te_f in zip(ue_splits, tr_splits, te_splits):

        scores = torch.mm(ue_f, i_emb.t())

        test_items = torch.from_numpy(te_f.todense()).float().to(device)
        non_train_items = torch.from_numpy(1-(tr_f.todense())).float().to(device)
        scores = scores * non_train_items

        _, test_indices = torch.topk(scores, dim=1, k=k)
        
        pred_items = torch.zeros_like(scores).float()
        pred_items.scatter_(dim=1, index=test_indices, src=torch.ones_like(test_indices).float().to(device))

        topk_preds = torch.zeros_like(scores).float()
        topk_preds.scatter_(dim=1, index=test_indices[:, :k], src=torch.ones_like(test_indices).float())
        
        TP = (test_items * topk_preds).sum(1)                      
        rec = TP/test_items.sum(1)
   
        ndcg = compute_ndcg_k(pred_items, test_items, test_indices, k)

        recall_k.append(rec)
        ndcg_k.append(ndcg)

    return torch.cat(ndcg_k).mean(), torch.cat(recall_k).mean()

# 5. 학습

In [9]:
make_graph_data_set = MakeGraphDataSet(config = config)
ngcf_adj_matrix = make_graph_data_set.get_ngcf_adj_matrix_data()
R_train, R_valid, R_total = make_graph_data_set.get_R_data()

In [35]:
model = LightGCN(
    n_users = make_graph_data_set.num_user,
    n_items = make_graph_data_set.num_item,
    emb_dim = config.emb_dim,
    n_layers = config.n_layers,
    reg = config.reg,
    node_dropout = config.node_dropout,
    adj_mtx = ngcf_adj_matrix,
    ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

Initializing weights...
Weights initialized.


In [36]:
best_hit = 0
for epoch in range(1, config.num_epochs + 1):
    tbar = tqdm(range(1))
    for _ in tbar:
        train_loss = train(
            model = model, 
            make_graph_data_set = make_graph_data_set, 
            optimizer = optimizer,
            n_batch = config.n_batch,
            )
        with torch.no_grad():
            ndcg, hit = evaluate(
                u_emb = model.u_final_embeddings.detach(), 
                i_emb = model.i_final_embeddings.detach(), 
                Rtr = R_train, 
                Rte = R_valid, 
                k = 10,
                )

        if best_hit < hit:
            best_hit = hit
            torch.save(model.state_dict(), os.path.join(config.model_path, config.model_name))

        tbar.set_description(f'Epoch: {epoch:3d}| Train loss: {train_loss:.5f}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

Epoch:   1| Train loss: 0.69310| NDCG@10: 0.15678| HIT@10: 0.03682: 100%|██████████| 1/1 [01:42<00:00, 102.55s/it]
Epoch:   2| Train loss: 0.69277| NDCG@10: 0.30541| HIT@10: 0.08269: 100%|██████████| 1/1 [01:40<00:00, 100.12s/it]
Epoch:   3| Train loss: 0.69145| NDCG@10: 0.31800| HIT@10: 0.08658: 100%|██████████| 1/1 [01:41<00:00, 101.38s/it]
Epoch:   4| Train loss: 0.68792| NDCG@10: 0.32011| HIT@10: 0.08687: 100%|██████████| 1/1 [01:41<00:00, 101.42s/it]
Epoch:   5| Train loss: 0.68066| NDCG@10: 0.32013| HIT@10: 0.08704: 100%|██████████| 1/1 [01:41<00:00, 101.72s/it]
Epoch:   6| Train loss: 0.66885| NDCG@10: 0.32091| HIT@10: 0.08708: 100%|██████████| 1/1 [01:41<00:00, 101.79s/it]
Epoch:   7| Train loss: 0.65215| NDCG@10: 0.32087| HIT@10: 0.08689: 100%|██████████| 1/1 [01:41<00:00, 101.27s/it]
Epoch:   8| Train loss: 0.63180| NDCG@10: 0.31983| HIT@10: 0.08689: 100%|██████████| 1/1 [01:40<00:00, 100.42s/it]
Epoch:   9| Train loss: 0.60826| NDCG@10: 0.32082| HIT@10: 0.08701: 100%|███████

KeyboardInterrupt: 

# 6. 예측

In [37]:
model.load_state_dict(torch.load(os.path.join(config.model_path, config.model_name)))
submision = []
with torch.no_grad():
    k = 10
    u_emb = model.u_final_embeddings.detach()
    i_emb = model.i_final_embeddings.detach()

    scores = torch.mm(u_emb, i_emb.t())

    test_items = torch.from_numpy(R_total.todense()).float().to(device)
    non_train_items = torch.from_numpy(1-(R_total.todense())).float().to(device)
    scores = scores * non_train_items

    _, test_indices = torch.topk(scores, dim=1, k=k)

    pred_items = torch.zeros_like(scores).float()
    pred_items.scatter_(dim=1, index=test_indices, src=torch.ones_like(test_indices).float().to(device))
    pred_items = pred_items.argsort(dim = 1)
    for user, item_list in enumerate(pred_items):
        item_list = item_list[-10:].cpu().numpy().tolist()
        for item in item_list:
            submision.append(
                {   
                    'user' : make_graph_data_set.user_decoder[user],
                    'item' : make_graph_data_set.item_decoder[item],
                }
            )

submision = pd.DataFrame(submision)


In [38]:
submision.to_csv(os.path.join(config.submission_path, config.submission_name), index=False)

In [39]:
submision

,user,item
0,11,8961
1,11,4886
2,11,858
3,11,1704
4,11,50
...,...,...
313595,138493,2762
313596,138493,4226
313597,138493,858
313598,138493,50
